<a href="https://colab.research.google.com/github/angiekim05/hwp_to_csv/blob/main/%ED%95%9C%EA%B8%80_%EB%AC%B8%EC%84%9C_%EC%B7%A8%ED%95%A9_%EC%9E%90%EB%8F%99%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install olefile
import olefile
import os
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for olefile: filename=olefile-0.46-py2.py3-none-any.whl size=35432 sha256=c091562416849ac8f550194c465e8dfbb1f6b49c49256b86deef19dd4d0110c7
  Stored in directory: /root/.cache/pip/wheels/02/39/c0/9eb1f7a42b4b38f6f333b6314d4ed11c46f12a0f7b78194f0d
Successfully built olefile


In [3]:
path = "/content/학술대회참가신청서.hwp"
f = olefile.OleFileIO(path)
encoded_txt = f.openstream("PrvText").read()
text = encoded_txt.decode("utf-16",errors="ignore")
print(text)



ㅇㅇㅇ학회
학술대회 참가신청서

<성  명><홍길동>
<소 속 (학생: 학교)><ㅇㅇ대학교><직 위 (학생: 학위과정)><석사과정>
<연 락 처><(휴대폰)><010-0000-0000><(이메일) ><sample@gmail.com>
<참가형태><□ 학술대회><식사여부 (필수)><0월0일><□ 만찬><□ Workshop><0월0일><□ 조식><□ 학술대회 + Workshop><□ 중식>
<참가구분><□ 일반참가자><□ 학생참가자>
<회원여부><□ 평생회원   ><□ 정회원><□ 준회원><□ 비회원>
<입금정보><입금자><홍길동><입금액><0만원>
<개인정보 제공 및 활용 동의 ><상기 개인정보를 ooo학회 학술대회 관련 참가신청 관리와 운영 업무에 활용하는 것에 동의합니다.  □ 동의함                       >
<보안서약서><상기 본인은 ooo학회 학술대회의 내용을  동의없이 녹음 및 녹화하지 않을 것임을 서약합니다.  □ 서약함>
<   상기 본인은 ㅇㅇㅇ학회 학술대회에 참가 신청합니다.  20  .        .        .                             신청인:     홍 길 동     (서명생략)  ㅇㅇㅇ학회 귀중>



In [10]:
class HWPfile():
  def __init__(self,path):
    self.table = self.hwp_table(path)
    self.datas = self.text2data(self.table)
    
  def print_datas(self):
    print(*self.table[:-1],sep="\n")

  def hwp_table(self,path):
    f = olefile.OleFileIO(path)
    encoded_txt = f.openstream("PrvText").read()
    text = encoded_txt.decode("utf-16",errors="ignore")
    
    text = text.replace(" ","").replace("><","\t").replace(">","") 
    text = text.split("\r\n")
    table = [t.replace("<","").split("\t") for t in text if len(t) > 0 and t[0] == "<"]
    table[7]=table[7][1][-4:] # 동의함 부분만 추출
    table[8]=table[8][1][-4:] # 서약함 부분만 추출
    return table

  def text2data(self,table):
    datas = []
    datas.append(table[0][1])
    datas.append(table[1][1])
    datas.append(table[1][3])
    datas.append(table[2][2])  
    datas.append(table[2][4])

    datas += [""]+[False]*3+[""]*2

    for param in table[3]:
      if param[0] != "□":

        # 참가형태는 3가지 중 1개        
        if "학술대회+Workshop" in param:
          datas[5]="학술대회 + Workshop"
        elif "학술대회" in param:
          datas[5]="학술대회"
        elif "Workshop" in param:
          datas[5]="Workshop"
        
        # 식사는 중복 가능
        elif "만찬" in param:
          datas[6]=True
        elif "조식" in param:
          datas[7]=True
        elif "중식" in param:
          datas[8]=True

    for i,name in zip([1,2],["일반참가자","학생참가자"]):
      if table[4][i][0] != "□":
        datas[9]=name
        break
        
    for i,name in zip([1,2,3,4],["평생회원","정회원","준회원","비회원"]):
      if table[5][i][0] != "□":
        datas[10]=name
        break
        
    datas.append(table[6][2])

    money = table[6][4]
    if "만" in money:
      money = money[:-2]
    else:
      money = int(money[:-1].replace(",",""))//10000
    datas.append(str(money)+"만원")

    if table[7][0] != "□":
      datas.append("동의")
    else:
      datas.append("동의안함")
    if table[7][0] != "□":
      datas.append("서약함")
    else:
      datas.append("서약안함")
    return datas

In [11]:
# 파일 리스트
file_list = [name for name in os.listdir('.') if name[-3:]=="hwp"]
file_list

# 문서 내 데이터
datas = []
for path in file_list:
  data = HWPfile(path)
  datas.append(data.datas)
  data.print_datas()

['성명', '홍길동']
['소속(학생:학교)', 'ㅇㅇ대학교', '직위(학생:학위과정)', '석사과정']
['연락처', '(휴대폰)', '010-0000-0000', '(이메일)', 'sample@gmail.com']
['참가형태', '□학술대회', '식사여부(필수)', '0월0일', '□만찬', '□Workshop', '0월0일', '□조식', '□학술대회+Workshop', '□중식']
['참가구분', '□일반참가자', '□학생참가자']
['회원여부', '□평생회원', '□정회원', '□준회원', '□비회원']
['입금정보', '입금자', '홍길동', '입금액', '0만원']
□동의함
□서약함


In [12]:
# dataframe에 담아 csv로 저장
cols = ['성명', '소속', '직위', '휴대폰', '이메일', '참가형태', '만찬', '조식', '중식', '참가구분', '회원여부', '입금자', '입금액',"동의","서약"]
df = pd.DataFrame(datas, columns=cols)
df

,성명,소속,직위,휴대폰,이메일,참가형태,만찬,조식,중식,참가구분,회원여부,입금자,입금액,동의,서약
0,홍길동,ㅇㅇ대학교,석사과정,010-0000-0000,sample@gmail.com,,False,False,False,,,홍길동,0만원,동의안함,서약안함


In [ ]:
# df.to_csv("sample.csv")